## Introduction - Suicide Data Analysis

The dataset contains the following features:

1. **Country**: The country where the suicide statistics were recorded.

2. **Age**: Age groups of the population, categorizing individuals into different ranges.

3. **Sex**: Gender of individuals, indicating whether the suicide statistics are for males or females.

4. **Suicides Number**: The absolute count of suicides recorded.

5. **Population**: The total population of the specified demographic group.

6. **Suicides/100k Pop**: The suicide rate per 100,000 population, providing a normalized measure.

7. **GDP for Year ($)**: The Gross Domestic Product (GDP) for the given year in dollars.

8. **GDP per Capita ($)**: The GDP per capita, calculated as GDP divided by the population.

9. **Generation**: A generational cohort classification, grouping individuals based on birth years.


# Install pyspark

In [31]:
!pip install pyspark

In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, when, regexp_replace, desc, sum, max, min, asc, to_date
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [33]:
spark = SparkSession.builder.appName("SuicideDataAnalysis").getOrCreate()

In [34]:
file_path = 'C:/Users/madih/big_data_analytics_project/dataset/suicide_statistics.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [35]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |-- gdp_for_year ($): string (nullable = true)
 |-- gdp_per_capita ($): integer (nullable = true)
 |-- generation: string (nullable = true)



In [36]:
df.show(5)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+----------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year|gdp_for_year ($)|gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+----------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|        NULL|   2,156,624,900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|        NULL|   2,156,624,900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|        NULL|   2,156,624,900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4.59| Albania

In [37]:
df.describe().show()

+-------+----------+------------------+------+-----------+------------------+------------------+------------------+--------------+-------------------+-----------------+------------------+----------+
|summary|   country|              year|   sex|        age|       suicides_no|        population| suicides/100k pop|  country-year|       HDI for year| gdp_for_year ($)|gdp_per_capita ($)|generation|
+-------+----------+------------------+------+-----------+------------------+------------------+------------------+--------------+-------------------+-----------------+------------------+----------+
|  count|     27820|             27820| 27820|      27820|             27820|             27820|             27820|         27820|               8364|            27820|             27820|     27820|
|   mean|      NULL|2001.2583752695903|  NULL|       NULL|242.57440690150972|1844793.6173975556|12.816097411933894|          NULL| 0.7766011477761785|             NULL|16866.464414090584|      NULL|
| std

# Data Transformation

In [38]:
df = df.withColumn("gdp_for_year ($)", regexp_replace(col("gdp_for_year ($)"), ",", ""))

In [39]:
df.show(5)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+----------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year|gdp_for_year ($)|gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+----------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|        NULL|      2156624900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|        NULL|      2156624900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|        NULL|      2156624900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4.59| Albania

In [40]:
df = df.withColumn("gdp_for_year ($)", df["gdp_for_year ($)"].cast('long'))
df.show(5)

+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+----------------+------------------+---------------+
|country|year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|HDI for year|gdp_for_year ($)|gdp_per_capita ($)|     generation|
+-------+----+------+-----------+-----------+----------+-----------------+------------+------------+----------------+------------------+---------------+
|Albania|1987|  male|15-24 years|         21|    312900|             6.71| Albania1987|        NULL|      2156624900|               796|   Generation X|
|Albania|1987|  male|35-54 years|         16|    308000|             5.19| Albania1987|        NULL|      2156624900|               796|         Silent|
|Albania|1987|female|15-24 years|         14|    289700|             4.83| Albania1987|        NULL|      2156624900|               796|   Generation X|
|Albania|1987|  male|  75+ years|          1|     21800|             4.59| Albania

In [41]:
df = df.withColumn("year", to_date(col("year")))

In [42]:
df.printSchema()

root
 |-- country: string (nullable = true)
 |-- year: date (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- suicides_no: integer (nullable = true)
 |-- population: integer (nullable = true)
 |-- suicides/100k pop: double (nullable = true)
 |-- country-year: string (nullable = true)
 |-- HDI for year: double (nullable = true)
 |-- gdp_for_year ($): long (nullable = true)
 |-- gdp_per_capita ($): integer (nullable = true)
 |-- generation: string (nullable = true)



# Handle null values

In [44]:
# Check for null values in the entire DataFrame
null_counts = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns])

# Display null counts for each column
null_counts.show()

+-------+----+---+---+-----------+----------+-----------------+------------+------------+----------------+------------------+----------+
|country|year|sex|age|suicides_no|population|suicides/100k pop|country-year|HDI for year|gdp_for_year ($)|gdp_per_capita ($)|generation|
+-------+----+---+---+-----------+----------+-----------------+------------+------------+----------------+------------------+----------+
|      0|   0|  0|  0|          0|         0|                0|           0|       19456|               0|                 0|         0|
+-------+----+---+---+-----------+----------+-----------------+------------+------------+----------------+------------------+----------+



In [45]:
# drop 'HDI for year' as there are more than 50% null values
df = df.drop("HDI for year")
df.show(5)

+-------+----------+------+-----------+-----------+----------+-----------------+------------+----------------+------------------+---------------+
|country|      year|   sex|        age|suicides_no|population|suicides/100k pop|country-year|gdp_for_year ($)|gdp_per_capita ($)|     generation|
+-------+----------+------+-----------+-----------+----------+-----------------+------------+----------------+------------------+---------------+
|Albania|1987-01-01|  male|15-24 years|         21|    312900|             6.71| Albania1987|      2156624900|               796|   Generation X|
|Albania|1987-01-01|  male|35-54 years|         16|    308000|             5.19| Albania1987|      2156624900|               796|         Silent|
|Albania|1987-01-01|female|15-24 years|         14|    289700|             4.83| Albania1987|      2156624900|               796|   Generation X|
|Albania|1987-01-01|  male|  75+ years|          1|     21800|             4.59| Albania1987|      2156624900|              

# Data Analysis

In [46]:
correlation_matrix = df.stat.corr("suicides_no", "population")
correlation_matrix

0.6161622675219286

In [47]:
correlation_matrix = df.stat.corr("suicides_no", "gdp_for_year ($)")
correlation_matrix

0.4300958523435763

In [48]:
correlation_matrix = df.stat.corr("suicides_no", "gdp_per_capita ($)")
correlation_matrix

0.061329748840245356

In [49]:
country_group = df.groupBy("country").agg({"suicides_no": "sum", "population": "sum"})
country_group.show()

+-----------------+----------------+---------------+
|          country|sum(suicides_no)|sum(population)|
+-----------------+----------------+---------------+
|         Paraguay|            4783|      124641969|
|       Cabo Verde|              42|         452179|
|           Sweden|           37795|      253297131|
|         Kiribati|              53|         741606|
|Republic of Korea|          261730|     1354944936|
|           Guyana|            3426|       16594589|
|      Philippines|           21330|     1065068179|
|        Singapore|           10089|       94242441|
|             Fiji|             304|        8187391|
|           Turkey|           10131|      482640582|
|          Germany|          291262|     2024875612|
|         Maldives|              20|        2900246|
|           France|          329127|     1670756775|
|           Greece|           12368|      316096105|
|        Sri Lanka|           55641|      182525626|
|         Dominica|               0|          

In [50]:
avg_suicides_by_generation = df.groupBy("generation").agg({"suicides/100k pop": "avg"}).show()
avg_suicides_by_generation

+---------------+----------------------+
|     generation|avg(suicides/100k pop)|
+---------------+----------------------+
|   Generation X|    10.556874219725358|
|   Generation Z|    0.6422993197278909|
|     Millenials|     5.383596851471596|
|         Silent|    18.418848208673793|
|G.I. Generation|    23.946377551020426|
|        Boomers|    14.742094188376745|
+---------------+----------------------+



In [51]:
# Group by country and calculate the total number of suicides for each country
suicides_by_country = df.groupBy("country").agg(sum("suicides_no").alias("total_suicides"))

# Find the country with the maximum number of suicides
max_suicides_country = suicides_by_country.select("country", "total_suicides").orderBy(col("total_suicides").desc()).first()

print(f"The country with the maximum number of suicides is: {max_suicides_country['country']} with {max_suicides_country['total_suicides']} suicides.")

The country with the maximum number of suicides is: Russian Federation with 1209742 suicides.


In [52]:
# Group by country and calculate the total number of suicides for each country
suicides_by_country = df.groupBy("country").agg(sum("suicides_no").alias("total_suicides"))

# Find the country with the minimum number of suicides
min_suicides_country = suicides_by_country.select("country", "total_suicides").orderBy(col("total_suicides").asc()).first()

print(f"The country with the minimum number of suicides is: {min_suicides_country['country']} with {min_suicides_country['total_suicides']} suicides.")

The country with the minimum number of suicides is: Dominica with 0 suicides.


In [53]:
# Calculate the total number of suicides by age group
total_suicides_by_age = df.groupBy("age").agg(sum("suicides_no").alias("total_suicides")).orderBy("age")

# Show the result
total_suicides_by_age.show()

+-----------+--------------+
|        age|total_suicides|
+-----------+--------------+
|15-24 years|        808542|
|25-34 years|       1123912|
|35-54 years|       2452141|
| 5-14 years|         52264|
|55-74 years|       1658443|
|  75+ years|        653118|
+-----------+--------------+



In [54]:
# Calculate the total number of suicides by gender
total_suicides_by_gender = df.groupBy("sex").agg(sum("suicides_no").alias("total_suicides")).collect()

# Print the results
for row in total_suicides_by_gender:
    gender = row["sex"]
    total_suicides = row["total_suicides"]
    print(f"Total suicides for {gender}: {total_suicides}")

Total suicides for female: 1559510
Total suicides for male: 5188910


# Build Linear Regression Model

In [55]:
feature_columns = ["gdp_for_year ($)", "population"]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
assembled_data = assembler.transform(df).select("features", "suicides_no")

In [56]:
train_data, test_data = assembled_data.randomSplit([0.8, 0.2], seed=123)

In [57]:
lr = LinearRegression(featuresCol="features", labelCol="suicides_no")
lr_model = lr.fit(train_data)

In [58]:
predictions = lr_model.transform(test_data)

In [59]:
evaluator = RegressionEvaluator(labelCol="suicides_no", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

In [60]:
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")

Root Mean Squared Error (RMSE) on test data: 903.4076168222439


In [61]:
# Evaluate the model using R2 score
evaluator_r2 = RegressionEvaluator(labelCol="suicides_no", predictionCol="prediction", metricName="r2")
r2_score = evaluator_r2.evaluate(predictions)

print(f"R2 Score on test data: {r2_score}")

R2 Score on test data: 0.3120584833823522
